RCA

In [1]:
from dipy.io.streamline import load_trk
from dipy.io.image import load_nifti
import numpy as np
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from dipy.tracking import streamline
import copy
import pickle
import math
import simnibs
from simnibs import sim_struct, run_simnibs
from simnibs.msh import (surface,
                        read_msh)
import random

In [2]:
# from stimvis.tms_stimulation import (my_deriv,
#                                     load_elems,
#                                     get_field,
#                                     get_ttrd,
#                                     deriv_e_field,
#                                     simulation)

def my_deriv(t,line,h=1):
    if t==0:
        return (-3*line[t]+4*line[t+1]-line[t+2])/(2*h)
    elif t==len(line)-1:
        return (line[t-2]-4*line[t-1]+3*line[t])/(2*h)
    else:
        return (line[t+1]-line[t-1])/(2*h)

def load_elems(nodes,elems):

    import numpy as np

    elems = elems[elems[:,3]!= -1,:]
    # Computing rectangles
    tmp = nodes[elems-1,:]
    elems_min = tmp.min(axis=1)
    elems_max = tmp.max(axis=1)
    tmp = 0
    sizes = (elems_max-elems_min).max(axis=0)
    # It is the index to reduce the elements to check
    order_min = np.argsort(elems_min[:,0])
    return {"Nodes":nodes, "Elems":elems, "El_min":elems_min, "El_max":elems_max, "Sizes":sizes,"Order_min":order_min}

# a function to get e-field vector at a given position [x,y,z]
def get_field(ttt, point, my_field):
    best_ttt=get_ttrd(ttt,point)
    return np.dot(my_field[best_ttt['Nodes']-1].T, best_ttt['Weights'])

def get_ttrd(loaded_elems,point):
    import numpy as np
    # Just to use names I have used before
    nodes = loaded_elems["Nodes"]
    elems = loaded_elems["Elems"]
    elems_min = loaded_elems["El_min"]
    elems_max = loaded_elems["El_max"]
    sizes = loaded_elems["Sizes"]
    order_min = loaded_elems["Order_min"]
    
    # Binary search to reduce the iterating points from 4mln to around 200k.
    r = np.searchsorted(elems_min[:,0],point[0],side='left',sorter=order_min)
    l = np.searchsorted(elems_min[:,0],point[0] - sizes[0],side='right',sorter=order_min)
    # Projection the data to only these points
    e_max = elems_max[order_min[l:r]]
    e_min = elems_min[order_min[l:r]]
    
    # Checks which ttrds are possible to contain the point
    potential_ttrds = order_min[l:r][(point[0] <= e_max[:,0]) & (e_min[:,1]<= point[1]) & (point[1] <= e_max[:,1]) & (e_min[:,2]<= point[2]) & (point[2] <= e_max[:,2])]
    
    # It checks if the ttrd contains the point
    def check_ttrd(ttrd, point):
        coord = np.column_stack((ttrd[1,:]-ttrd[0,:],ttrd[2,:]-ttrd[0,:],ttrd[3,:]-ttrd[0,:]))
        coord = np.linalg.inv(coord).dot(point-ttrd[0,:])
        return coord.min() >= 0 and coord.sum() <= 1
    # It checks if the ttrd with num ttrdNum contains the point
    def check_ttrd_byNum(ttrdNum, point):
        ttrd = nodes[elems[ttrdNum]-1]
        return check_ttrd(ttrd,point)
    
    # Just takes all ttrds that contain points
    nodeIndices = elems[[x for x in potential_ttrds if check_ttrd_byNum(x,point)]][0]; 
    ns = nodes[nodeIndices-1]

    norms = np.sum((ns-point)**2,axis=-1)**0.5
    weights = 1/(norms+1e-10)
    weights = weights / weights.sum()
    
    return {"Nodes":nodeIndices,"Weights":weights}


# a function to calculate directional derivatives of the effective field at a given point [x,y,z]
def deriv_e_field(coordinates, e_field_nodes, LSD, ttt):
   
    step=0.05

    x1=coordinates[0]
    y1=coordinates[1]
    z1=coordinates[2]
    x0=coordinates[0]-step
    x2=coordinates[0]+step
    y0=coordinates[1]-step
    y2=coordinates[1]+step
    z0=coordinates[2]-step
    z2=coordinates[2]+step

    f_x0_y1_z1=np.dot(get_field(ttt,np.asarray([x0,y1,z1]), e_field_nodes), LSD)
    f_x2_y1_z1=np.dot(get_field(ttt,np.asarray([x2,y1,z1]), e_field_nodes), LSD)
    f_x1_y1_z1=np.dot(get_field(ttt,np.asarray([x1,y1,z1]), e_field_nodes), LSD)
    f_x1_y0_z1=np.dot(get_field(ttt,np.asarray([x1,y0,z1]), e_field_nodes), LSD)
    f_x1_y2_z1=np.dot(get_field(ttt,np.asarray([x1,y2,z1]), e_field_nodes), LSD)
    f_x1_y1_z0=np.dot(get_field(ttt,np.asarray([x1,y1,z0]), e_field_nodes), LSD)
    f_x1_y1_z2=np.dot(get_field(ttt,np.asarray([x1,y1,z2]), e_field_nodes), LSD)
    
    gradx=my_deriv(1,[f_x0_y1_z1,f_x1_y1_z1,f_x2_y1_z1], step)
    grady=my_deriv(1,[f_x1_y0_z1,f_x1_y1_z1,f_x1_y2_z1], step)
    gradz=my_deriv(1,[f_x1_y1_z0,f_x1_y1_z1,f_x1_y1_z2], step)
    
    return np.dot([gradx, grady, gradz], LSD)

In [3]:
# Ввод данных
tractography_file='../Data/tracts.trk'

sft = load_trk(tractography_file, tractography_file)
streams = sft.streamlines
streams_array = np.asarray(streams)
print('imported tractography data:'+tractography_file)

fname_T1 = '../Data/T1fs_conform.nii.gz'#input("Please, specify the T1fs_conform image that has been generated during head meshing procedure. ")
data_T1, affine_T1 = load_nifti(fname_T1)

fname_FA = '../Data/dti_fa.nii' #input("Please, specify the FA image. ")
data_FA, affine_FA = load_nifti(fname_FA)
print('loaded T1fs_conform.nii and FA images')

mesh_path = '../Data/subject.msh' #input("Please, specify the head mesh file. ")

last_slach = max([i for i, ltr in enumerate(mesh_path) if ltr == '/'])+1
subject_name = mesh_path[last_slach:-4]


# specify the directory where you would like to save your simulation results
# input example:'/home/Example_data/Output'
out_dir = '../Output' #input("Please, specify the directory where you would like to save your simulation results. ")

imported tractography data:../Data/tracts.trk
loaded T1fs_conform.nii and FA images


In [5]:
%%time
# Co-registration of T1fs_conform and FA images. Performed in 4 steps.
# Step 1. Calculation of the center of mass transform. Used later as starting transform.
c_of_mass = transform_centers_of_mass(data_T1, affine_T1, data_FA, affine_FA)
print('calculated c_of_mass transformation')

# Step 2. Calculation of a 3D translation transform. Used in the next step as starting transform.
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(data_T1, data_FA, transform, params0,
                    affine_T1, affine_FA,
                    starting_affine=starting_affine)
print('calculated 3D translation transform')

# Step 3. Calculation of a Rigid 3D transform. Used in the next step as starting transform
transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(data_T1, data_FA, transform, params0, 
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)
print('calculated Rigid 3D transform')

# Step 4. Calculation of an affine transform. Used for co-registration of T1 and FA images.
transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(data_T1, data_FA, transform, params0,
                         affine_T1, affine_FA,
                         starting_affine=starting_affine)

print('calculated Affine 3D transform')

identity = np.eye(4)

inv_affine_FA = np.linalg.inv(affine_FA)
inv_affine_T1 = np.linalg.inv(affine_T1)
inv_affine = np.linalg.inv(affine.affine)

# transforming streamlines to FA space
new_streams_FA = streamline.transform_streamlines(streams, inv_affine_FA)
new_streams_FA_array = np.asarray(new_streams_FA)

T1_to_FA = np.dot(inv_affine_FA, np.dot(affine.affine, affine_T1))
FA_to_T1 = np.linalg.inv(T1_to_FA)

# transforming streamlines from FA to T1 space
new_streams_T1 = streamline.transform_streamlines(new_streams_FA, FA_to_T1)
new_streams_T1_array = np.asarray(new_streams_T1)

# calculating amline derivatives along the streamlines to get the local orientation of the streamlines
streams_array_derivative = copy.deepcopy(new_streams_T1_array)

print('calculating streamline derivatives')
for stream in range(len(new_streams_T1_array)):
    my_steam = new_streams_T1_array[stream]
    for t in range(len(my_steam[:, 0])):
        streams_array_derivative[stream][t, 0] = my_deriv(t, my_steam[:, 0])
        streams_array_derivative[stream][t, 1] = my_deriv(t, my_steam[:, 1])
        streams_array_derivative[stream][t, 2] = my_deriv(t, my_steam[:, 2])
        deriv_norm = np.linalg.norm(streams_array_derivative[stream][t, :])
        streams_array_derivative[stream][t, :] = streams_array_derivative[stream][t, :]/deriv_norm    
        
#------ Чтоб не повторять по нескольку раз и немного экономить время (потом удалить)
with open('../new_streams_T1_array.txt', 'wb') as f:
    pickle.dump(new_streams_T1_array, f)
f.close()
with open('../streams_array_derivative.txt', 'wb') as f:
    pickle.dump(streams_array_derivative, f)
f.close()


calculated c_of_mass transformation
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated 3D translation transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Rigid 3D transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Affine 3D transform
calculating streamline derivatives
CPU times: user 20min 25s, sys: 37.4 s, total: 21min 2s
Wall time: 16min 19s


In [4]:
with open('../new_streams_T1_array.txt', 'rb') as f:
    new_streams_T1_array = pickle.load(f)
f.close()

with open('../streams_array_derivative.txt', 'rb') as f:
    streams_array_derivative = pickle.load(f)
f.close()

In [5]:
mesh_struct = read_msh(mesh_path)
scalp=surface.Surface(mesh_struct, [5,1005])

def get_coords_from_angles(alfa, beta):
    # (1) Преобразование в трехмерную систему координат. x,y,z ∈ [0,255]
    r=256/2
    a= math.radians(alfa)
    b= math.radians(beta)
    
    x= r * math.sin(a) * math.cos(b) +r
    y= r * math.cos(a) +r
    z= r * math.sin(a) * math.sin(b) +r
    
    coords=np.array((x,y,z))
    coords-=256/2
    
    # (2) Получение проекции координаты на голову
    proj_coords,_=scalp.projectPoint(coords)
    return proj_coords

In [6]:
# Ограничение в 25мм
x_min = np.min(new_streams_T1_array[0][:,0]) - 25 - 256/2
x_max = np.max(new_streams_T1_array[0][:,0]) + 25 - 256/2
y_min = np.min(new_streams_T1_array[0][:,1]) - 25 - 256/2
y_max = np.max(new_streams_T1_array[0][:,1]) + 25 - 256/2
z_min = np.min(new_streams_T1_array[0][:,2]) - 25 - 256/2
z_max = np.max(new_streams_T1_array[0][:,2]) + 25 - 256/2
print (x_min, x_max, y_min, y_max, z_min, z_max)

-58.35510653248957 6.999714416924746 -37.13407266408484 29.310656427633916 -31.340430584812097 93.1118822930465


In [7]:
def simulation(fnamehead, pathfem, pos_centre=[-74.296158, -10.213354, 28.307243], 
               pos_ydir=[-74.217369, -37.293205, 20.05232], distance=4, current_change=1e6):
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False
    
    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'

    # Initialize a coil position
    pos = tmslist.add_position()
    pos.centre = pos_centre # Place the coil over
    pos.pos_ydir = pos_ydir # Point the coil towards
    pos.distance = distance # Distance between coil and head
    pos.didt = current_change # Rate of change of current in the coil, in A/s.
    
    run_simnibs(s) 

    
def simulation_list(fnamehead, pathfem, pos_list, distance=4, current_change=1e6, cores=1):
    """It differs from the usual simulation in that it simulates several points.
    It can also run simulations using multiple cores.

    pos_list example: [(pos_centre, pos_ydir), (pos_centre2, pos_ydir2)]
    """
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False

    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'

    for coords,y_dir in pos_list:
        pos = tmslist.add_position()
        pos.centre = coords # Place the coil over
        pos.pos_ydir = y_dir # Point the coil towards
        pos.distance = distance # Distance between coil and head
        pos.didt = current_change

    run_simnibs(s, cpus=cores) 

In [8]:
class Point:
    def __init__(self, angles): #alfa,beta,rotation
        self.angles=angles
        self.coords=get_coords_from_angles(angles[0], angles[1]).tolist()
        
    def __lt__(self, other):
        return self.effect > other.effect
         
    def calc_effect(self, mesh_path, current_out_dir):
        l1 = 2  # membrane space constant 2mm
        l2 = l1**2

        field_mesh = read_msh(mesh_path)
        field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
        field_at_nodes = field_as_nodedata.value
        ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)

        effective_field = new_streams_T1_array.copy()
        s_a_derivative=streams_array_derivative.copy()
        effective_field -= 256/2

        for stream, my_stream in enumerate(effective_field):
            print('starting _'+str(stream)+' out of '+str(len(effective_field)))
            for t, xyz in enumerate(my_stream):
                field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

                component_1 = l1*np.dot(field_vector_xyz, s_a_derivative[stream][t, :])
                component_2 = l2*deriv_e_field(xyz, field_at_nodes, s_a_derivative[stream][t, :], ttt)

                effective_field[stream][t]=np.array([component_1,
                                                     component_2,
                                                     component_1 + component_2])
        x,y,z=self.coords
        current_out_dir+=str(x)+'_'+str(y)+'_'+str(z)
        with open(current_out_dir+'_effective_field.txt', 'wb') as f:
            pickle.dump(effective_field, f)
        f.close()

        # Поиск максимума среди всех точек:
        self.effect= max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])
        return self.effect
    
    def is_point_close_to_streams(self):
        return (self.coords[0]>=x_min and self.coords[0]<=x_max 
                and self.coords[1]>=y_min and self.coords[1]<=y_max 
                and self.coords[2]>=z_min and self.coords[2]<=z_max)
    
    def get_pos_ydir(self):
        rotation_angle = math.radians(self.angles[2])
        self.pos_ydir = get_coords_from_angles(self.angles[0] + math.sin(rotation_angle)*7, # just 7
                                               self.angles[1] + math.cos(rotation_angle)*7)
        return self.pos_ydir

----
## Выполнение RCA
1. Последовательное
2. Параллельное

### Последовательное выполнение

In [9]:
search_area={'alfa':[0,180], 'beta':[0,180], 'rotation':[0,360]}
number_of_trials = 10
last_steps_start_with = 7
number_in_trial = 10
n_best = 5


from ipywidgets import IntProgress
from IPython.display import display
prg_bar = IntProgress(min = 0, max = number_of_trials) # Создаем прогрессбар
display(prg_bar)

IntProgress(value=0, max=10)

In [10]:
best_point=None
points=[]
SA_list=[]

for current_trial in range(number_of_trials):
    # Create points
    # The points on the list are unique by coordinates and are close to nerve streams
    points.clear()
    if current_trial < last_steps_start_with:
        # All points are from search area
        i=0
        while len(points)<number_in_trial:
            point=Point([random.uniform(*search_area['alfa']),
                         random.uniform(*search_area['beta']),
                         random.uniform(*search_area['rotation'])])
            if (point.is_point_close_to_streams()
                    and point.coords not in [x.coords for x in points]):
                points.append(point)
    else:
        # 80% of points takes from search area
        while len(points)<int(number_in_trial*0.8):
            point=Point([random.uniform(*search_area['alfa']),
                         random.uniform(*search_area['beta']),
                         random.uniform(*search_area['rotation'])])
            if (point.is_point_close_to_streams()
                    and point.coords not in [x.coords for x in points]):
                points.append(point)
        # 20% of points takes from initial search area
        while len(points)<number_in_trial:
            point=Point([random.uniform(0, 180),
                         random.uniform(0, 180),
                         random.uniform(0, 360)])
            if (point.is_point_close_to_streams()
                    and point.coords not in [x.coords for x in points]):
                points.append(point)
    
    # Calculation of effects
    #simulation
    sim_out_dir=out_dir+'/Step_'+str(current_trial)
    pos_list = [(point.coords, point.get_pos_ydir()) for point in points]
    simulation_list(mesh_path, sim_out_dir, pos_list)
    #effect calculation
    for i, point in enumerate(points):
        point.calc_effect(sim_out_dir+'/subject_TMS_1-{0:04}_Magstim_70mm_Fig8_nii_scalar.msh'.format(i+1),
                          sim_out_dir+'/Effects')
    
    
    # Override search area
    # sort points by effect
    points.sort()
    if best_point is None or best_point.effect<points[0].effect:
        best_point=points[0]
    for point in points[:n_best]:
        print(point.effect, point.coords)
    search_area['alfa'] = [min(points[:n_best], key=lambda point : point.angles[0]).angles[0]-3, #key=alfa
                           max(points[:n_best], key=lambda point : point.angles[0]).angles[0]+3]
    search_area['beta'] = [min(points[:n_best], key=lambda point : point.angles[1]).angles[1]-3, #key=beta
                           max(points[:n_best], key=lambda point : point.angles[1]).angles[1]+3]
    search_area['rotation'] = [min(points[:n_best], key=lambda point : point.angles[2]).angles[2]-3, #key=rotation
                               max(points[:n_best], key=lambda point : point.angles[2]).angles[2]+3]
    
    prg_bar.value+=1 # prg_bar
    SA_list.append(copy.deepcopy(search_area))
    print('Search area:', search_area)
print('RESULT:', best_point.coords, best_point.effect)

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/Step_0
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/Step_0
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/Step_0
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/Step_0
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil fil

[ simnibs ]INFO: Time to solve system: 10.93s
INFO:simnibs:Time to solve system: 10.93s
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 36 Residual Norm: 1.34e-10
INFO:simnibs:Number of iterations: 36 Residual Norm: 1.34e-10
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 10.67s
INFO:simnibs:Time to solve system: 10.67s
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of ite

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.54e+00 V/m |1.13e+00 V/m |6.24e-01 V/m |
|normE |1.54e+00 V/m |1.13e+00 V/m |6.24e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.94e+03 mm³ |1.75e+04 mm³ |
|normE |4.94e+03 mm³ |1.75e+04 mm³ |

subject_TMS_1-0002_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.74e+00 V/m |1.20e+00 V/m |6.79e-01 V/m |
|normE |1.74e+00 V/m |1.20e+00 V/m |6.79e-01 V/m |

Field Focality
--------

[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/Step_0/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/Step_0/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/Step_0/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/Step_0/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/Step_0/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/Step_0/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: /home/roman/Документы/GitHub/Ou

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 o

...

[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/Step_9
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/Step_9
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/Step_9
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/Step_9
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz


[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 36 Residual Norm: 1.33e-10
INFO:simnibs:Number of iterations: 36 Residual Norm: 1.33e-10
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 10.57s
INFO:simnibs:Time to solve system: 10.57s
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 6.97e-11
INFO:simnibs:Number of iterations: 37 Residual Norm:

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.61e+00 V/m |1.10e+00 V/m |6.34e-01 V/m |
|normE |1.61e+00 V/m |1.10e+00 V/m |6.34e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |3.71e+03 mm³ |1.48e+04 mm³ |
|normE |3.71e+03 mm³ |1.48e+04 mm³ |

subject_TMS_1-0002_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.67e+00 V/m |1.15e+00 V/m |6.61e-01 V/m |
|normE |1.67e+00 V/m |1.15e+00 V/m |6.61e-01 V/m |

Field Focality
--------

[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/Step_9/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/Step_9/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/Step_9/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/Step_9/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/Step_9/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/Step_9/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: /home/roman/Документы/GitHub/Ou

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 o

### Параллельное выполнение

In [7]:
import multiprocessing
from os.path import exists

In [10]:
class Consumer(multiprocessing.Process):

    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue

    def run(self):
        pname = self.name
            
        while True:
            point = self.task_queue.get()
            
            if point is None: # poison pill
                print('Exiting %s...' % pname)
                self.task_queue.task_done()
                break
            
            print('{0} processing task: {1:04}'.format(pname, point.id))
            sim_out_dir=out_dir+'/Step_'+str(point.step)
            point.calc_effect(sim_out_dir+'/subject_TMS_1-{0:04}_Magstim_70mm_Fig8_nii_scalar.msh'.format(point.id),
                          sim_out_dir+'/Effects')
            
            self.task_queue.task_done()
            self.result_queue.put(point)

----

In [10]:
# Settings:
search_area={'alfa':[0,180], 'beta':[0,180], 'rotation':[0,360]}
number_of_trials = 10 #20
last_steps_start_with = 7 #14
number_in_trial = 10
n_best = 5

# Progress bar
from ipywidgets import IntProgress
from IPython.display import display
prg_bar2 = IntProgress(min = 0, max = number_of_trials)
display(prg_bar2)

IntProgress(value=0, max=10)

In [45]:
best_point=None
points=[]
SA_list=[]

for current_trial in range(number_of_trials):
    # Creation of points
    # The points on the list are unique by coordinates and are close to nerve streams
    points.clear()
    if current_trial < last_steps_start_with:
        # All points are from search area
        while len(points)<number_in_trial:
            point=Point([random.uniform(*search_area['alfa']),
                         random.uniform(*search_area['beta']),
                         random.uniform(*search_area['rotation'])])
            if (point.is_point_close_to_streams()
                    and point.coords not in [x.coords for x in points]):
                points.append(point)
                
    else:
        # 80% of points takes from search area
        while len(points)<int(number_in_trial*0.8):
            point=Point([random.uniform(*search_area['alfa']),
                         random.uniform(*search_area['beta']),
                         random.uniform(*search_area['rotation'])])
            if (point.is_point_close_to_streams()
                    and point.coords not in [x.coords for x in points]):
                points.append(point)
        # 20% of points takes from initial search area
        while len(points)<number_in_trial:
            point=Point([random.uniform(0, 180),
                         random.uniform(0, 180),
                         random.uniform(0, 360)])
            if (point.is_point_close_to_streams()
                    and point.coords not in [x.coords for x in points]):
                points.append(point)
    
    # multiprocessing calculation
    #simulations
    sim_out_dir=out_dir+'/Step_'+str(step)
    pos_list = [(point.coords, point.get_pos_ydir()) for point in points]
    simulation_list(mesh_path, sim_out_dir, pos_list, cores=2)
    #effects
    if __name__ == '__main__':
        tasks = multiprocessing.JoinableQueue()
        results = multiprocessing.Queue()
        # spawning consumers
        n_consumers = 2 #multiprocessing.cpu_count()
        print('Spawning %i consumers...' % n_consumers)
        consumers = [Consumer(tasks, results) for i in range(n_consumers)]

        for consumer in consumers:
            consumer.start()

        # enqueueing jobs
        for i, point in enumerate(points):
            point.step=step
            point.id=i+1
            tasks.put(point)
        # poison pill
        for i in range(n_consumers):
            tasks.put(None)
        
#         for consumer in consumers:
#             consumer.join()
        tasks.join()
        step_points=[results.get() for i in range(len(points))]

    step_points.sort()
    if best_point is None or best_point.fitness<step_points[0].fitness:
        best_point=step_points[0]
    # Override search area
    # sort points by effect
    for point in step_points[:n_best]:
        print(point.fitness, point.coords)
    search_area['alfa'] = [min(step_points[:n_best], key=lambda point : point.angles[0]).angles[0], #key=alfa
                           max(step_points[:n_best], key=lambda point : point.angles[0]).angles[0]]
    search_area['beta'] = [min(step_points[:n_best], key=lambda point : point.angles[1]).angles[1], #key=beta
                           max(step_points[:n_best], key=lambda point : point.angles[1]).angles[1]]
    search_area['rotation'] = [min(step_points[:n_best], key=lambda point : point.angles[2]).angles[2], #key=rotation
                               max(step_points[:n_best], key=lambda point : point.angles[2]).angles[2]]
    
    prg_bar2.value+=1 # prg_bar
    SA_list.append(search_area) # delit it !!
    print('Search area:', search_area)
print('RESULT:', best_point.coords, best_point.fitness)

Spawning 2 consumers...
Consumer-10 processing task: [-10.472664, -20.784859, 87.332367]
Consumer-9 processing task: [-23.548613, -2.556196, 81.883247]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-23.548613_-2.556196_81.883247
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-23.548613_-2.556196_81.883247
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-10.472664_-20.784859_

INFO:simnibs:Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_-10.472664_-20.784859_87.332367/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/simulation_at_pos_-10.472664_-20.784859_87.332367/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/simulation_at_pos_-10.472664_-20.784859_87.332367/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]SUMMARY: 
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of t

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-10 processing task: [-9.105796, -18.291948, 87.477325]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-9.105796_-18.291948_87.477325
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-9.105796_-18.291948_87.477325
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-9.105796_-18.291948_87.477325
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-9.105796_-18.291948_87.477325
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS sim

...

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-18.254286_-15.367653_84.652832
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-18.254286_-15.367653_84.652832
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-18.254286_-15.367653_84.652832
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-18.254286_-15.367653_84.652832
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS

Exiting Consumer-15...


[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[ 5.4297e-01  7.8126e-01  3.0793e-01 -1.9486e+01]
 [ 8.3290e-01 -5.4777e-01 -7.8877e-02 -1.5052e+01]
 [ 1.0705e-01  2.9930e-01 -9.4813e-01  8.8445e+01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
INFO:simnibs:Matsimnibs: 
[[ 5.4297e-01  7.8126e-01  3.0793e-01 -1.9486e+01]
 [ 8.3290e-01 -5.4777e-01 -7.8877e-02 -1.5052e+01]
 [ 1.0705e-01  2.9930e-01 -9.4813e-01  8.8445e+01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 10.07s to assemble FEM matrix
INFO:simnibs:10.07s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-16...
5.805172734598524 [-21.974909, -12.7256, 83.123322]
5.626655899700262 [-21.095423, -15.035339, 83.625511]
5.596419626163229 [-18.869629, -14.291732, 84.363319]
5.405788298457805 [-19.831064, -15.802687, 84.17701]
5.249761519719881 [-17.047865, -16.150974, 85.123421]
Search area: {'alfa': [94.46677680648502, 95.85084836393978], 'beta': [103.86601102007879, 106.18622378559311], 'rotation': [156.53050774081706, 161.53973415878042]}


KeyboardInterrupt: 

# <center>ТЕСТЫ<center/>
1. Выполнение (симуляция+эффект)
2. Только симуляция
3. Только эффект

### Тест на скорость выполнения 10 симуляций с разным количеством задействованных ядер
- Without multiprocessing: 22min 55s
- 1 core: 23min 10s
- 2 cores: 15min 13s
- 3 cores: 19min 4s, 18min, 59s
- 4 cores: browser dropped

In [12]:
%%time
# Without multiprocessing
result=[]

my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]

result=[calc_data_for_point(x) for x in my_input]

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

...

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
CPU times: user 38min 28s, sys: 2min 22s, total: 40min 51s
Wall time: 22min 55s


In [13]:
%%time
# An unsuccessful attempt to perform a parallel calculation
# Everything was done with a single core
# 23min 10s
result=[]

if __name__ == '__main__':
    manager = multiprocessing.Manager()
    results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    #results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system
    n_consumers = multiprocessing.cpu_count()
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
        
    tasks.join()
    print('-----')
    result =[results.get() for i in range(len(my_input))]

    print('Done.')
print('Code, please be faster')

Spawning 4 consumers...
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


...

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
-----
Done.
Code, please be faster
CPU times: user 305 ms, sys: 181 ms, total: 485 ms
Wall time: 23min 10s


Using a poison pill

In [42]:
%%time
# mulriprocessing 2 cores
result=[]

if __name__ == '__main__':
    #manager = multiprocessing.Manager()
    #results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system -1
    n_consumers = multiprocessing.cpu_count()-2
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)
    
    tasks.join()
    
    result =[results.get() for i in range(len(my_input))]
    print('Done.')

Spawning 2 consumers...
Consumer-67 processing task:
Consumer-68 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_68.

INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]SUMMARY: 
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles o

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-68 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_70.763947_41.499119_35.410679
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_70.763947_41.499119_35.410679


...

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

Consumer-68 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |7.08e-01 V/m |4.97e-01 V/m |2.95e-01 V/m |
|normE |7.08e-01 V/m |4.97e-01 V/m |2.95e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.08e+03 mm³ |1.80e+04 mm³ |
|normE |4.08e+03 mm³ |1.80e+04 mm³ |

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.1144

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Exiting Consumer-67...
Exiting Consumer-68...
Done.
CPU times: user 372 ms, sys: 193 ms, total: 565 ms
Wall time: 15min 13s


In [41]:
%%time
# mulriprocessing 3 cores
result=[]

if __name__ == '__main__':
    #manager = multiprocessing.Manager()
    #results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system -1
    n_consumers = multiprocessing.cpu_count()-1
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)
    
    tasks.join()
    
    result =[results.get() for i in range(len(my_input))]
    print('Done.')

Spawning 3 consumers...
Consumer-64 processing task:
Consumer-65 processing task:
Consumer-66 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
INFO:simnibs:Subject Path: None
INFO:simnibs:Subject Path: None
[ simni

INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Time to prepare the KSP: 2.98s
INFO:simnibs:Time to prepare the KSP: 2.98s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Time to prepare the KSP: 3.20s
INFO:simnibs:Time to prepare the KSP: 3.20s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.68e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 8.68e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 26.29s
INFO:simnibs:Time to solve system: 26.29s
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.56e-11
INFO:simnibs:Number of iterations: 37 Re

[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.196838_59.625603_54.552277/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_46.566605_51.393372_55.326584/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_37.326508_67.819153_43.353844/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_38.196838_59.625603_54.552277/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:=====================================
[ simnibs ]INFO: =====================================
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
INFO:simnibs:=====================================


starting _0 out of 6
starting _1 out of 6
starting _0 out of 6
starting _0 out of 6
starting _2 out of 6
starting _1 out of 6
starting _3 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _4 out of 6
starting _3 out of 6
starting _3 out of 6
starting _5 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-65 processing task:


...

Exiting Consumer-65...


[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_53.455193_45.403191_53.438599
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_53.455193_45.403191_53.438599
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities


Exiting Consumer-66...


[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[ -0.2375  -0.6192   0.7485 -50.7337]
 [  0.8817   0.186    0.4336 -67.8709]
 [ -0.4077   0.7629   0.5017 -59.7529]
 [  0.       0.       0.       1.    ]]
INFO:simnibs:Matsimnibs: 
[[ -0.2375  -0.6192   0.7485 -50.7337]
 [  0.8817   0.186    0.4336 -67.8709]
 [ -0.4077   0.7629   0.5017 -59.7529]
 [  0.       0.       0.       1.    ]]
[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 10.05s to assemble FEM matrix
INFO:simnibs:10.05s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-64...
Done.
CPU times: user 406 ms, sys: 199 ms, total: 606 ms
Wall time: 19min 4s


In [45]:
for x in result:
    print(x)

((88.18419787246357, 20.73120179621355), array([68.045219, 37.519661, 42.394608]), 'hello')
((2.789068869577316, 81.56864381120418), array([40.139816, 76.252769, 27.867176]), 'hello')
((85.89709256243536, 2.793913185425856), array([70.763947, 41.499119, 35.410679]), 'hello')
((72.56203220579968, 16.70041975214285), array([67.101227, 45.456612, 37.561089]), 'hello')
((89.4216826413991, 53.29433385839758), array([48.788033, 38.685085, 62.084385]), 'hello')
((45.467058293190746, 65.85102939695857), array([35.645798, 57.389046, 59.302292]), 'hello')
((31.773505699008155, 22.175446896186447), array([51.413162, 61.83654 , 39.104778]), 'hello')
((17.37969850234252, 35.37628364866059), array([43.610199, 73.678207, 28.99267 ]), 'hello')
((99.38871034032253, 21.77568286797914), array([69.260773, 32.842316, 43.797318]), 'hello')
((58.41724328575197, 19.53173674151335), array([59.426521, 52.856895, 40.114452]), 'hello')


In [48]:
%%time
# mulriprocessing 3 cores (Test 2)
result=[]

if __name__ == '__main__':
    #manager = multiprocessing.Manager()
    #results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system -1
    n_consumers = multiprocessing.cpu_count()-1
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)
    
    tasks.join()
    
    result =[results.get() for i in range(len(my_input))]
    print('Done.')

Spawning 3 consumers...
Consumer-75 processing task:
Consumer-74 processing task:
Consumer-73 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_54.768379_69.078339_23.323191
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Simulat

INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Time to prepare the KSP: 3.11s
INFO:simnibs:Time to prepare the KSP: 3.11s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.67e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 8.67e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 25.94s
INFO:simnibs:Time to solve system: 25.94s
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.60e-11
INFO:simnibs:Number of iterations: 37 Resid

INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_31.82081_61.335064_57.417877/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.82081_61.335064_57.417877/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_31.82081_61.335064_57.417877/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================


starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-73 processing task:


...

Exiting Consumer-75...


[ simnibs ]INFO: 11.72s to assemble FEM matrix
INFO:simnibs:11.72s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field


Exiting Consumer-73...


DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 4.29s
INFO:simnibs:Time to prepare the KSP: 4.29s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.52e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 8.52e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 11.75s
INFO:simnibs:Time to solve system: 11.75s
[ simnibs ]INFO: Creating visualization

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-74...
Done.
CPU times: user 406 ms, sys: 186 ms, total: 591 ms
Wall time: 18min 59s


---
### Test
Проведение симуляции по нескольким точкам может быть проведено параллельно.
Необходимо узнать каким образом можно быстрее провести вычисление. Под вычислением подразумевается проведение двух последовательных действий: провести симуляцию, рассчитать эффект.
Варианты проведения этих действий:
* L -> L
* L -> P
* P -> L
* P -> P
* P(L->L)

Буква **L** обозначает линейное выполнение, а **P** - параллельное. Последний вариант подразумевает под собой параллельное выполнение двух действий. Другими словами, один процесс проводит симуляцию и рассчет эффекта по одной точке, а парралельно от него выполняет те же действия другой процесс по другой точке.


#### Test 1 Just simulation without effect calculation
* simulations one by one - 9min 52s
* all in one simulation (1 core) - 9min 15s
* 2 cores - 7min 3s
* 3 cores - 7min
* 4 cores - 7min 5s

In [15]:
def simulation(fnamehead, pathfem, pos_list=None, pos_centre=[-74.296158, -10.213354, 28.307243], pos_ydir=[-74.217369, -37.293205, 20.05232], distance=4, current_change=1e6, cores=1):
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False
    
    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'
    
    if pos_list:
        for coords,y_dir in pos_list:
            pos = tmslist.add_position()
            pos.centre = coords # Place the coil over
            pos.pos_ydir = y_dir # Point the coil towards
            pos.distance = distance # Distance between coil and head
            pos.didt = current_change
    else:
        # Initialize a coil position
        pos = tmslist.add_position()
        pos.centre = pos_centre # Place the coil over
        pos.pos_ydir = pos_ydir # Point the coil towards
        pos.distance = distance # Distance between coil and head
        pos.didt = current_change # Rate of change of current in the coil, in A/s.
    
    run_simnibs(s) 

In [16]:
def angles_to_coords (alfa, beta):
    r=256/2
    a= math.radians(alfa)
    b= math.radians(beta)
    
    x= r * math.sin(a) * math.cos(b) +r
    y= r * math.cos(a) +r
    z= r * math.sin(a) * math.sin(b) +r
    
    coords=np.array((x,y,z))
    coords-=256/2
    return coords

In [17]:
# create points for tests
coords_list=[angles_to_coords(*[random.uniform(0,180) for i in range(2)])
             for number in range(12)]
points=[]
for coords in coords_list:
    proj_coords,normal=scalp.projectPoint(coords)
    pos_y=copy.deepcopy(proj_coords)
    if np.abs(normal[1]) < .8:
        pos_y[1] += 10
    else:
        pos_y[0] += 10
    points.append([proj_coords, pos_y])

---

In [8]:
%%time
# Test 1.1: (9min 52s)
# one by one
for coords, y_dir in points:
    x,y,z=coords
    current_out_dir = out_dir+str(x)+'_'+str(y)+'_'+str(z)
    simulation(mesh_path, current_out_dir, pos_centre=coords, pos_ydir=y_dir)

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-23.848312_-78.351585_60.389214
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-23.848312_-78.351585_60.389214
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-23.848312_-78.351585_60.389214
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-23.848312_-78.351585_60.389214
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS

INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 2.36s
INFO:simnibs:Time to prepare the KSP: 2.36s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 5.17e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 5.17e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 11.55s
INFO:simnibs:Time to solve system: 11.55s
[ simnibs ]INFO: Creating visualizations
INFO:simnibs:Creating visualizations
[ simnibs ]SUMMARY: 
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field 

INFO:simnibs:Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_31.813528_-86.241692_38.874073/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/GitHub/Output/simulation_at_pos_31.813528_-86.241692_38.874073/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/GitHub/Output/simulation_at_pos_31.813528_-86.241692_38.874073/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head

[ simnibs ]INFO: Matsimnibs: 
[[-3.5518e-01 -5.8843e-02  9.3294e-01 -7.1804e+01]
 [ 3.4694e-18  9.9802e-01  6.2948e-02 -1.5432e+01]
 [-9.3480e-01  2.2358e-02 -3.5448e-01  4.8531e+01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
INFO:simnibs:Matsimnibs: 
[[-3.5518e-01 -5.8843e-02  9.3294e-01 -7.1804e+01]
 [ 3.4694e-18  9.9802e-01  6.2948e-02 -1.5432e+01]
 [-9.3480e-01  2.2358e-02 -3.5448e-01  4.8531e+01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 9.99s to assemble FEM matrix
INFO:simnibs:9.99s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boo

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.48e+00 V/m |9.80e-01 V/m |5.79e-01 V/m |
|normE |1.48e+00 V/m |9.80e-01 V/m |5.79e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |3.12e+03 mm³ |1.40e+04 mm³ |
|normE |3.12e+03 mm³ |1.40e+04 mm³ |

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_-10.111234_-104.23922_11.667373/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_-10.111234_-104.23922_11.667373

[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-38.243923_-97.287743_-1.081758
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-38.243923_-97.287743_-1.081758
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[-6.9389e-18 

INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 12.17s
INFO:simnibs:Time to solve system: 12.17s
[ simnibs ]INFO: Creating visualizations
INFO:simnibs:Creating visualizations
[ simnibs ]SUMMARY: 
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.56e+00 V/m |1.05e+00 V/m |5.91e-01 V/m |
|normE |1.56e+00 V/m |1.05e+00 V/m |5.91e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |3.46e+03 mm³ |1.51e+04 mm³ |
|normE |3.46e+03 mm³ |1.51e+04 mm³ |

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)

[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-2.597751_66.073013_58.230518
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_-2.597751_66.073013_58.230518
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-2.597751_66.073013_58.230518
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_-2.597751_66.073013_58.230518
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Run

INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 2.40s
INFO:simnibs:Time to prepare the KSP: 2.40s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 4.35e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 4.35e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 12.12s
INFO:simnibs:Time to solve system: 12.12s
[ 

CPU times: user 22min 14s, sys: 1min 43s, total: 23min 58s
Wall time: 9min 52s


In [11]:
%%time
# Test 1.2: (9min 11s)
# all in one simulation
if True:
    current_out_dir = out_dir+'all'
    simulation(mesh_path, current_out_dir, pos_list=points)

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simn

[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 10.13s to assemble FEM matrix
INFO:simnibs:10.13s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 2.39s
INFO:simnibs:Time to prepare the KSP: 2.39s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 5.82e-11
INFO:simnibs:Number of iterations: 37

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.20e+00 V/m |8.57e-01 V/m |4.91e-01 V/m |
|normE |1.20e+00 V/m |8.57e-01 V/m |4.91e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.49e+03 mm³ |1.62e+04 mm³ |
|normE |4.49e+03 mm³ |1.62e+04 mm³ |

subject_TMS_1-0002_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.55e+00 V/m |1.13e+00 V/m |6.39e-01 V/m |
|normE |1.55e+00 V/m |1.13e+00 V/m |6.39e-01 V/m |

Field Focality
--------

INFO:simnibs:Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0004_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0005_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0006_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0007_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0008_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all/subject_TMS_1-0009_Magstim_70mm_Fig

CPU times: user 15min 42s, sys: 1min 22s, total: 17min 5s
Wall time: 9min 11s


In [18]:
%%time
# Test 1.3: (7min 3s) 5m11s
# 2 cores
if True:
    current_out_dir = out_dir+'all_2'
    simulation(mesh_path, current_out_dir, pos_list=points, cores=2)

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_2
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_2
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_2
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_2
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packa

[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 7.49s to assemble FEM matrix
INFO:simnibs:7.49s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 2.47s
INFO:simnibs:Time to prepare the KSP: 2.47s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 6.96e-11
INFO:simnibs:Number of iterations: 37 R

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.59e+00 V/m |1.14e+00 V/m |6.28e-01 V/m |
|normE |1.59e+00 V/m |1.14e+00 V/m |6.28e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.57e+03 mm³ |1.62e+04 mm³ |
|normE |4.57e+03 mm³ |1.62e+04 mm³ |

subject_TMS_1-0002_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.45e+00 V/m |1.06e+00 V/m |6.20e-01 V/m |
|normE |1.45e+00 V/m |1.06e+00 V/m |6.20e-01 V/m |

Field Focality
--------

INFO:simnibs:Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0004_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0005_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0006_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0007_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-0008_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_2/subject_TMS_1-000

CPU times: user 13min 34s, sys: 56.9 s, total: 14min 31s
Wall time: 5min 11s


In [19]:
%%time
# Test 1.4: (7min) 5m7s
# 3 cores
if True:
    current_out_dir = out_dir+'all_3'
    simulation(mesh_path, current_out_dir, pos_list=points, cores=3)

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_3
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_3
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_3
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_3
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packa

[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 7.15s to assemble FEM matrix
INFO:simnibs:7.15s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 2.08s
INFO:simnibs:Time to prepare the KSP: 2.08s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 6.96e-11
INFO:simnibs:Number of iterations: 37 R

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.59e+00 V/m |1.14e+00 V/m |6.28e-01 V/m |
|normE |1.59e+00 V/m |1.14e+00 V/m |6.28e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.57e+03 mm³ |1.62e+04 mm³ |
|normE |4.57e+03 mm³ |1.62e+04 mm³ |

subject_TMS_1-0002_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.45e+00 V/m |1.06e+00 V/m |6.20e-01 V/m |
|normE |1.45e+00 V/m |1.06e+00 V/m |6.20e-01 V/m |

Field Focality
--------

INFO:simnibs:Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0004_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0005_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0006_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0007_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-0008_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_3/subject_TMS_1-000

CPU times: user 13min 36s, sys: 58.3 s, total: 14min 34s
Wall time: 5min 7s


In [20]:
%%time
# Test 1.5: (7min 5s) 5m1s
# 4 cores
if True:
    current_out_dir = out_dir+'all_4_2'
    simulation(mesh_path, current_out_dir, pos_list=points, cores=4)

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/GitHub/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2
INFO:simnibs:Simulation Folder: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/si

[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 6.63s to assemble FEM matrix
INFO:simnibs:6.63s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 1.98s
INFO:simnibs:Time to prepare the KSP: 1.98s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 6.96e-11
INFO:simnibs:Number of iterations: 37 R

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.59e+00 V/m |1.14e+00 V/m |6.28e-01 V/m |
|normE |1.59e+00 V/m |1.14e+00 V/m |6.28e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.57e+03 mm³ |1.62e+04 mm³ |
|normE |4.57e+03 mm³ |1.62e+04 mm³ |

subject_TMS_1-0002_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.45e+00 V/m |1.06e+00 V/m |6.20e-01 V/m |
|normE |1.45e+00 V/m |1.06e+00 V/m |6.20e-01 V/m |

Field Focality
--------

INFO:simnibs:Result Files:
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0002_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0003_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0004_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0005_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0006_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0007_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2/subject_TMS_1-0008_Magstim_70mm_Fig8_nii_scalar.msh
/home/roman/Документы/GitHub/Output/simulation_at_pos_all_4_2

CPU times: user 13min 28s, sys: 52.2 s, total: 14min 20s
Wall time: 5min 1s


#### Test 2. Расчет эффекта
* последовательно - 19min 26s
* 2 cores - 
* 3 cores - 
* 4 cores - 

In [8]:
def calc_only_TMS_effects(mesh_path, current_out_dir, number):
    # Быстрей на ничтожные секунды, но может суммарно и наберется значимое ускорение во времени
    l1 = 2  # membrane space constant 2mm
    l2 = l1**2

    field_mesh = read_msh(mesh_path)
    field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
    field_at_nodes = field_as_nodedata.value
    ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)

    effective_field = new_streams_T1_array.copy()
    s_a_derivative=streams_array_derivative.copy()
    effective_field -= 256/2
    
    for stream, my_stream in enumerate(effective_field):
        print('starting _'+str(stream)+' out of '+str(len(effective_field)))
        for t, xyz in enumerate(my_stream):
            field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

            component_1 = l1*np.dot(field_vector_xyz, s_a_derivative[stream][t, :])
            component_2 = l2*deriv_e_field(xyz, field_at_nodes, s_a_derivative[stream][t, :], ttt)
            
            effective_field[stream][t]=np.array([component_1,
                                                 component_2,
                                                 component_1 + component_2])
            
    with open(current_out_dir+'/'+str(number)+'_' + subject_name+'_effective_field.txt', 'wb') as f:
        pickle.dump(effective_field, f)
    f.close()

    # Поиск максимума среди всех точек:
    return max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])

In [9]:
class Consumer(multiprocessing.Process):

    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue

    def run(self):
        pname = self.name
            
        while True:
            i = self.task_queue.get()
            
            if i is None: # poison pill
                print('Exiting %s...' % pname)
                self.task_queue.task_done()
                break
                
            print('{0} processing task: {1}'.format(pname, i))
            try:
                path_1=out_dir+'all_4/'+subject_name+'_TMS_1-'+ '{0:04}'.format(i) +'_Magstim_70mm_Fig8_nii_scalar.msh'
                calc_only_TMS_effects(path_1, out_dir+'all_4', i)
            except Exception as ex:
                print('ERROR!::', ex)
            
            self.result_queue.put(i)
            self.task_queue.task_done()

In [79]:
# Test на 12 подсчетов эффектов
# обычный, 2 ядра, 3 ядра, ну и (если смелый) на все 4 ядра.

In [12]:
%%time
# Test 2.1. Линейный расчет (19min 26s)
for i in range(1,13):
    path_1=out_dir+'all_4/'+subject_name+'_TMS_1-'+ '{0:04}'.format(i) +'_Magstim_70mm_Fig8_nii_scalar.msh'
    calc_only_TMS_effects(path_1, out_dir+'all_4', i)
    print('---------------')

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
---------------
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
---------------
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
---------------
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
---------------
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
---------------
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
---------------
starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
---------------
starti

In [10]:
%%time
# Test 2.2.
# Multiprocessing calculation. 2 cores (9min 28s) 16GB ОЗУ
if __name__ == '__main__':
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()
    # spawning consumers
    n_consumers = 2
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]

    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    for i in range(1,13):
        tasks.put(i)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)

    tasks.join()

Spawning 2 consumers...
Consumer-2 processing task: 2
Consumer-1 processing task: 1
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-1 processing task: 3
Consumer-2 processing task: 4
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-1 processing task: 5
Consumer-2 processing task: 6
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-2 processing task: 7
Consumer-1

In [11]:
%%time
# Test 2.3.
# Multiprocessing calculation. 3 cores (6min 44s) 16GB ОЗУ
if __name__ == '__main__':
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()
    # spawning consumers
    n_consumers = 3
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]

    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    for i in range(1,13):
        tasks.put(i)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)

    tasks.join()

Spawning 3 consumers...
Consumer-3 processing task: 1
Consumer-4 processing task: 2
Consumer-5 processing task: 3
starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-3 processing task: 4
Consumer-5 processing task: 5
Consumer-4 processing task: 6
starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-4 processing task: 7
Consumer-3

In [13]:
%%time
# Test 2.4.
# Multiprocessing calculation. 4 cores (5min 40s) 16GB ОЗУ
if __name__ == '__main__':
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()
    # spawning consumers
    n_consumers = 4
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]

    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    for i in range(1,13):
        tasks.put(i)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)

    tasks.join()

Spawning 4 consumers...
Consumer-10 processing task: 1
Consumer-11 processing task: 2
Consumer-12 processing task: 3
Consumer-13 processing task: 4
starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-11 processing task: 5
Consumer-10 processing task: 6
Consumer-12 processing task: 7
Consumer-13 processing task: 8
starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _2 ou